In this module I wrap the ALS estimation and scoring with the scikit-learn base estimator to make it compatible with other scikit-learn modules such as cross-validation and grid-search. Have a look at [jobch's excellent blog post and implementation](https://towardsdatascience.com/recommending-github-repositories-with-google-bigquery-and-the-implicit-library-e6cce666c77), that's where I stole most of the code :).

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
from implicit.als import AlternatingLeastSquares
from implicit.utils import nonzeros
import os
# disable internal multithreading
%env MKL_NUM_THREADS=1

env: MKL_NUM_THREADS=1


In [2]:
os.chdir("/mnt/c//Users/simon.weiss/Documents/Freaky-Friday/recommender/recommender-lastfm/code")

In [3]:
lastfm = pd.read_table("../data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv", 
                         usecols=[0, 2, 3], 
                         names=["user", "artist", "plays"],
                         na_filter = False,
                         encoding = "utf-8")

In [4]:
lastfm["dist_user_plays"] = lastfm.loc[:, ["user", "artist"]].groupby("user").transform("count")
lastfm["dist_artist_plays"] = lastfm.loc[:, ["user", "artist"]].groupby("artist").transform("count")


In [5]:
# Subsetting the data to artists with more than 1 distinct play and users with more than 1 distinct play
data = lastfm[(lastfm["dist_user_plays"] > 60) & (lastfm["dist_artist_plays"] > 800)].reset_index()

In [6]:
data['user'] = data['user'].astype("category")
data['artist'] = data['artist'].astype("category")

In [7]:
def print_stats(df, user, item):
    n_users = np.int64(df.loc[:, user].drop_duplicates().count())
    n_artists = np.int64(df.loc[:, item].drop_duplicates().count())
    sparsity =  (1 - float(df.shape[0]) / float(n_users*n_artists)) * 100
    print("Number of Users: {}".format(n_users))
    print("Number of Artists: {}".format(n_artists))
    print("Sparsity: {:.8} %".format(str(sparsity)))

In [8]:
print_stats(data, "user", "artist")

Number of Users: 25524
Number of Artists: 3487
Sparsity: 98.81094 %


In [9]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

In [10]:
class UtilityMatrixTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, confidence=40):
        self.confidence = confidence
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        # TODO: abstract column names
        return coo_matrix((X['plays'].astype(float), 
                   (X['artist'].cat.codes.copy(), 
                    X['user'].cat.codes.copy()))) * self.confidence

In [11]:
class ALSEstimator(BaseEstimator, TransformerMixin):
    def __init__(self, factors=50,
                       regularization=0.01,
                       iterations=10,
                       filter_seen=True):
        self.factors = factors
        self.regularization = regularization
        self.iterations = iterations
        self.filter_seen = filter_seen
        
    def fit(self, X, y=None):
        self.model = AlternatingLeastSquares(factors=self.factors,
                                             regularization=self.regularization,
                                             iterations=self.iterations,
                                             dtype=np.float64,
                                             use_native=True)
        self.model.fit(X)
        if self.filter_seen:
            self.fit_X = X
        return self
        
    def predict(self, X, y=None):
        predictions = np.dot(self.model.item_factors, self.model.user_factors.T)
        if self.filter_seen:
            predictions[self.fit_X.nonzero()] = -99
        return predictions

In [ ]:
## from https://gist.github.com/qZhang88/415578d5918bf9a5b50fff8f13ad5187

def dcg_at_k(r, k, method=1):
    """Score is discounted cumulative gain (dcg)
    Relevance is positive real values.  Can use binary
    as the previous methods.
    Example from
    http://www.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf
    >>> r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
    >>> dcg_at_k(r, 1, 0)
    3.0
    >>> dcg_at_k(r, 1)
    7.0
    >>> dcg_at_k(r, 2, 0)
    5.0
    >>> dcg_at_k(r, 2)
    8.8927892607143733
    >>> dcg_at_k(r, 10, 0)
    9.6051177391888114
    >>> dcg_at_k(r, 11, 0)
    9.6051177391888114
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]
    Returns:
        Discounted cumulative gain
    """
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum((2**r - 1) / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, k, method=1):
    """Score is normalized discounted cumulative gain (ndcg)
    Relevance is positive real values.  Can use binary
    as the previous methods.
    Example from
    http://www.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf
    >>> r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
    >>> ndcg_at_k(r, 1, 0)
    1.0
    >>> r = [2, 1, 2, 0]
    >>> ndcg_at_k(r, 4, 0)
    0.9203032077642922
    >>> ndcg_at_k(r, 4)
    0.9514426589871553
    >>> ndcg_at_k([0], 1, 0)
    0.0
    >>> ndcg_at_k([1], 2, 0)
    1.0
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]
    Returns:
        Normalized discounted cumulative gain
    """
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
return dcg_at_k(r, k, method) / dcg_max

In [15]:
#https://gist.github.com/mblondel/7337391

def dcg_score(y_true, y_score, k=10, gains="exponential"):
    """Discounted cumulative gain (DCG) at rank k
    Parameters
    ----------
    y_true : array-like, shape = [n_samples]
        Ground truth (true relevance labels).
    y_score : array-like, shape = [n_samples]
        Predicted scores.
    k : int
        Rank.
    gains : str
        Whether gains should be "exponential" (default) or "linear".
    Returns
    -------
    DCG @k : float
    """
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])

    if gains == "exponential":
        gains = 2 ** y_true - 1
    elif gains == "linear":
        gains = y_true
    else:
        raise ValueError("Invalid gains option.")

    # highest rank is 1 so +2 instead of +1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)


def ndcg_score(y_true, y_score, k=10, gains="exponential"):
    """Normalized discounted cumulative gain (NDCG) at rank k
    Parameters
    ----------
    y_true : array-like, shape = [n_samples]
        Ground truth (true relevance labels).
    y_score : array-like, shape = [n_samples]
        Predicted scores.
    k : int
        Rank.
    gains : str
        Whether gains should be "exponential" (default) or "linear".
    Returns
    -------
    NDCG @k : float
    """
    best = dcg_score(y_true, y_true, k, gains)
    if best == 0:
        return 0    
    actual = dcg_score(y_true, y_score, k, gains)
    return actual / best

In [16]:
def get_col(Y, col):
    return np.squeeze(np.asarray(Y[:,col]))

def ndcg_score_matrix(Y_true, Y_score, k=10, gains="exponential"):
    score = 0.0
    n_users = Y_true.shape[1]
    for u in range(n_users):
        s = ndcg_score(get_col(Y_true, u), get_col(Y_score, u))
        score += s
    return score / n_users

In [17]:
from sklearn.model_selection import PredefinedSplit

class LeavePOutByGroup():
    def __init__(self, X, p=5, n_splits=2):
        self.X = X
        self.p = p
        self.n_splits = n_splits
        test_fold = self.X.groupby("user").cumcount().apply(lambda x: int(x / p) if x < (n_splits * p) else -1)
        self.s = PredefinedSplit(test_fold)

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits

    def split(self, X=None, y=None, groups=None):
        return self.s.split()

In [18]:
def ndcg_scorer(estimator, X_test):
    truth = UtilityMatrixTransformer(confidence=1).fit_transform(X_test).todense()
    predictions = estimator.predict(X_test)
    return ndcg_score_matrix(truth, predictions, k=10)

In [19]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [20]:
rec_pipeline = Pipeline([
        ('matrix', UtilityMatrixTransformer()),
        ('als', ALSEstimator()),
])

param_grid = [
    {
        'matrix__confidence': [40, 100],
        'als__factors': [20, 100],
        'als__regularization': [1e-2, 1e-4],
    }
]

In [21]:
shuffled_train_set = data.reindex(np.random.permutation(data.index)).sort_values("user")
grid_search = GridSearchCV(rec_pipeline, param_grid,
                           cv=LeavePOutByGroup(shuffled_train_set, p=5, n_splits=3),
                           scoring=ndcg_scorer, verbose=1)
grid_search.fit(shuffled_train_set)

Fitting 3 folds for each of 18 candidates, totalling 54 fits


/home/simon/miniconda3/envs/implicit/lib/python3.6/site-packages/ipykernel/__main__.py:23: RuntimeWarning: overflow encountered in power
/home/simon/miniconda3/envs/implicit/lib/python3.6/site-packages/ipykernel/__main__.py:54: RuntimeWarning: invalid value encountered in double_scalars
/home/simon/miniconda3/envs/implicit/lib/python3.6/site-packages/ipykernel/__main__.py:23: RuntimeWarning: overflow encountered in power
/home/simon/miniconda3/envs/implicit/lib/python3.6/site-packages/ipykernel/__main__.py:54: RuntimeWarning: invalid value encountered in double_scalars
/home/simon/miniconda3/envs/implicit/lib/python3.6/site-packages/ipykernel/__main__.py:23: RuntimeWarning: overflow encountered in power
/home/simon/miniconda3/envs/implicit/lib/python3.6/site-packages/ipykernel/__main__.py:54: RuntimeWarning: invalid value encountered in double_scalars
/home/simon/miniconda3/envs/implicit/lib/python3.6/site-packages/ipykernel/__main__.py:23: RuntimeWarning: overflow encountered in power

/home/simon/miniconda3/envs/implicit/lib/python3.6/site-packages/ipykernel/__main__.py:23: RuntimeWarning: overflow encountered in power
/home/simon/miniconda3/envs/implicit/lib/python3.6/site-packages/ipykernel/__main__.py:54: RuntimeWarning: invalid value encountered in double_scalars
/home/simon/miniconda3/envs/implicit/lib/python3.6/site-packages/ipykernel/__main__.py:23: RuntimeWarning: overflow encountered in power
/home/simon/miniconda3/envs/implicit/lib/python3.6/site-packages/ipykernel/__main__.py:54: RuntimeWarning: invalid value encountered in double_scalars
/home/simon/miniconda3/envs/implicit/lib/python3.6/site-packages/ipykernel/__main__.py:23: RuntimeWarning: overflow encountered in power
/home/simon/miniconda3/envs/implicit/lib/python3.6/site-packages/ipykernel/__main__.py:54: RuntimeWarning: invalid value encountered in double_scalars
/home/simon/miniconda3/envs/implicit/lib/python3.6/site-packages/ipykernel/__main__.py:23: RuntimeWarning: overflow encountered in power

GridSearchCV(cv=<__main__.LeavePOutByGroup object at 0x7fa92ce5aa90>,
       error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('matrix', UtilityMatrixTransformer(confidence=40)), ('als', ALSEstimator(factors=50, filter_seen=True, iterations=10, regularization=0.01))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'matrix__confidence': [40, 100], 'als__factors': [20, 50, 100], 'als__regularization': [0.01, 0.001, 0.0001]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=<function ndcg_scorer at 0x7fa91b95f598>, verbose=1)

In [141]:
grid_search.best_params_

{'als__factors': 20, 'als__regularization': 0.01, 'matrix__confidence': 40}

In [142]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

nan {'als__factors': 20, 'als__regularization': 0.01, 'matrix__confidence': 40}
nan {'als__factors': 20, 'als__regularization': 0.01, 'matrix__confidence': 100}
nan {'als__factors': 20, 'als__regularization': 0.001, 'matrix__confidence': 40}
nan {'als__factors': 20, 'als__regularization': 0.001, 'matrix__confidence': 100}
nan {'als__factors': 20, 'als__regularization': 0.0001, 'matrix__confidence': 40}
nan {'als__factors': 20, 'als__regularization': 0.0001, 'matrix__confidence': 100}
nan {'als__factors': 50, 'als__regularization': 0.01, 'matrix__confidence': 40}
nan {'als__factors': 50, 'als__regularization': 0.01, 'matrix__confidence': 100}
nan {'als__factors': 50, 'als__regularization': 0.001, 'matrix__confidence': 40}
nan {'als__factors': 50, 'als__regularization': 0.001, 'matrix__confidence': 100}
nan {'als__factors': 50, 'als__regularization': 0.0001, 'matrix__confidence': 40}
nan {'als__factors': 50, 'als__regularization': 0.0001, 'matrix__confidence': 100}
nan {'als__factors': 1